In [ ]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pathlib import Path

# Set random seeds for reproducibility
random_seed = 42
np.random.seed(random_seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

PROJECT_ROOT = Path('/content')
EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    PROJECT_ROOT = Path('..').resolve()
    EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    raise FileNotFoundError(f'Could not find esm_outputs directory at {EMBED_DIR}')

features_path = EMBED_DIR / 'esm_features.npy'
labels_path = EMBED_DIR / 'labels.npy'
print('Loading embeddings from', features_path)
X = np.load(features_path)
y = np.load(labels_path)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=random_seed
)

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Initialize the Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=random_seed,
    n_jobs=-1,
    class_weight='balanced'
)

# Training the Random Forest model
print("Training the Random Forest model...")
rf_model.fit(X_train, y_train)
print("Training completed.")

# Evaluate on the test set
print("Evaluating on the test set...")
y_pred = rf_model.predict(X_test)
y_pred_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class
test_accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')

# Calculate additional metrics
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

# Plot Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix, annot=True, fmt="d", cmap="Blues",
    xticklabels=['Negative', 'Positive'],
    yticklabels=['Negative', 'Positive']
)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Random Forest')
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))

# ROC Curve and AUC
plt.figure(figsize=(12, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - Random Forest')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall Curve
plt.figure(figsize=(12, 6))
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Random Forest')
plt.show()

In [ ]:
import torch
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pathlib import Path

random_seed = 42
np.random.seed(random_seed)

PROJECT_ROOT = Path('/content')
EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    PROJECT_ROOT = Path('..').resolve()
    EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    raise FileNotFoundError(f'Could not find esm_outputs directory at {EMBED_DIR}')

features_path = EMBED_DIR / 'esm_features.npy'
labels_path = EMBED_DIR / 'labels.npy'
print('Loading embeddings from', features_path)
X = np.load(features_path)
y = np.load(labels_path)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=random_seed
)

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Update model parameters to use GPU
use_gpu = torch.cuda.is_available()
tree_method = 'gpu_hist' if use_gpu else 'hist'
if not use_gpu:
    print('GPU not detected for XGBoost; using CPU tree_method="hist"')
params = {
    'objective': 'binary:logistic',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'eval_metric': 'auc',
    'random_state': random_seed,
    'scale_pos_weight': len(y_train) / np.sum(y_train == 1),
    'tree_method': tree_method
}

# Training the XGBoost model
print("Training the XGBoost model...")
xgb_model = xgb.train(params, dtrain, num_boost_round=100)
print("Training completed.")

# Evaluate on the test set
print("Evaluating on the test set...")
y_pred_prob = xgb_model.predict(dtest)
y_pred = (y_pred_prob >= 0.5).astype(int)

test_accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')

# Calculate additional metrics
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

# Plot Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix, annot=True, fmt="d", cmap="Blues",
    xticklabels=['Negative', 'Positive'],
    yticklabels=['Negative', 'Positive']
)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - XGBoost')
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))

# ROC Curve and AUC
plt.figure(figsize=(12, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - XGBoost')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall Curve
plt.figure(figsize=(12, 6))
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - XGBoost')
plt.show()


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from pathlib import Path

random_seed = 42
np.random.seed(random_seed)

PROJECT_ROOT = Path('/content')
EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    PROJECT_ROOT = Path('..').resolve()
    EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    raise FileNotFoundError(f'Could not find esm_outputs directory at {EMBED_DIR}')

features_path = EMBED_DIR / 'esm_features.npy'
labels_path = EMBED_DIR / 'labels.npy'
print('Loading embeddings from', features_path)
X = np.load(features_path)
y = np.load(labels_path)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=random_seed
)

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Initialize the SVM model
svm_model = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=random_seed)

# Training the SVM model
print("Training the SVM model...")
svm_model.fit(X_train, y_train)
print("Training completed.")

# Evaluate on the test set
y_pred = svm_model.predict(X_test)
y_pred_prob = svm_model.predict_proba(X_test)[:, 1]

# Calculate accuracy and AUC
test_accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - SVM')
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
plt.figure(figsize=(12, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - SVM')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
plt.figure(figsize=(12, 6))
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - SVM')
plt.show()


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, accuracy_score, confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from pathlib import Path

random_seed = 42
np.random.seed(random_seed)

PROJECT_ROOT = Path('/content')
EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    PROJECT_ROOT = Path('..').resolve()
    EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    raise FileNotFoundError(f'Could not find esm_outputs directory at {EMBED_DIR}')

features_path = EMBED_DIR / 'esm_features.npy'
labels_path = EMBED_DIR / 'labels.npy'
print('Loading embeddings from', features_path)
X = np.load(features_path)
y = np.load(labels_path)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=random_seed
)

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Initialize the Bagging model with Decision Tree as base estimator
bagging_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=50,
    random_state=random_seed,
    n_jobs=-1
)

# Training the Bagging model
print("Training the Bagging model...")
bagging_model.fit(X_train, y_train)
print("Training completed.")

# Evaluate on the test set
y_pred = bagging_model.predict(X_test)
y_pred_prob = bagging_model.predict_proba(X_test)[:, 1]

# Calculate accuracy and AUC
test_accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Bagging')
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
plt.figure(figsize=(12, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - Bagging')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
plt.figure(figsize=(12, 6))
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Bagging')
plt.show()



In [ ]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    confusion_matrix,
    classification_report,
)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression

random_seed = 42
np.random.seed(random_seed)

PROJECT_ROOT = Path('/content')
EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    PROJECT_ROOT = Path('..').resolve()
    EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    raise FileNotFoundError(f'Could not find esm_outputs directory at {EMBED_DIR}')

features_path = EMBED_DIR / 'esm_features.npy'
labels_path = EMBED_DIR / 'labels.npy'
print('Loading embeddings from', features_path)
X = np.load(features_path)
y = np.load(labels_path)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=random_seed
)

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

log_reg = LogisticRegression(max_iter=2000, class_weight='balanced', solver='lbfgs', n_jobs=-1)
print('Training Logistic Regression...')
log_reg.fit(X_train, y_train)
print('Training completed.')

y_pred = log_reg.predict(X_test)
y_pred_prob = log_reg.predict_proba(X_test)[:, 1]

test_accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')

conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Logistic Regression')
plt.show()

print('Classification Report:', classification_report(y_test, y_pred))

fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression')
plt.legend(loc='lower right')
plt.show()

precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
plt.figure(figsize=(6, 6))
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Logistic Regression')
plt.show()
